In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.callbacks import ModelCheckpoint

In [2]:
train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)
validation_datagen=ImageDataGenerator(rescale=1./255)

In [3]:
training_set=train_datagen.flow_from_directory(
    r"E:\Documents\Docs\Pradeesh\Academic\VESIT\CNN_project\OCT2017\trainnew",
    target_size=(128,128),
    class_mode="categorical",
    batch_size=64,
    shuffle=True
)
test_set=test_datagen.flow_from_directory(
    r"E:\Documents\Docs\Pradeesh\Academic\VESIT\CNN_project\OCT2017\testnew",
    target_size=(128,128),
    batch_size=64,
    class_mode="categorical",
    shuffle=True
)
validation_set=validation_datagen.flow_from_directory(
   r"E:\Documents\Docs\Pradeesh\Academic\VESIT\CNN_project\OCT2017\validation",
    target_size=(128,128),
    batch_size=64,
    class_mode="categorical",
    shuffle=True
)

Found 10000 images belonging to 4 classes.
Found 3112 images belonging to 4 classes.
Found 2500 images belonging to 4 classes.


In [4]:
# Load Xception pre-trained on ImageNet without the top (classification) layer
base_model = Xception(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(4, activation='softmax')(x)

In [5]:
# Create the final model
xception_model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
xception_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 63, 63, 32)           864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 63, 63, 32)           128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 63, 63, 32)           0         ['block1_conv1_bn[0][0]'] 

In [7]:
checkpoint_callback = ModelCheckpoint('xception_model_checkpoint_epoch_{epoch:02d}.h5',
                                       save_weights_only=True)

In [8]:
total_epochs = 25

In [ ]:
xception_model.fit(
    training_set, 
    validation_data=validation_set, 
    epochs=total_epochs,
    validation_steps=len(validation_set), 
    steps_per_epoch=len(training_set),
    callbacks= [checkpoint_callback]
)

Epoch 1/25


157/157 [==============================] - 1458s 9s/step - loss: 0.8350 - accuracy: 0.6689 - val_loss: 0.6795 - val_accuracy: 0.7304
Epoch 2/25
157/157 [==============================] - 1510s 10s/step - loss: 0.6307 - accuracy: 0.7504 - val_loss: 0.6233 - val_accuracy: 0.7464
Epoch 3/25
157/157 [==============================] - 1557s 10s/step - loss: 0.5695 - accuracy: 0.7795 - val_loss: 0.6242 - val_accuracy: 0.7456
Epoch 4/25
157/157 [==============================] - 3591s 23s/step - loss: 0.5237 - accuracy: 0.7958 - val_loss: 0.6053 - val_accuracy: 0.7568
Epoch 5/25
157/157 [==============================] - 1496s 10s/step - loss: 0.4972 - accuracy: 0.8048 - val_loss: 0.5475 - val_accuracy: 0.7752
Epoch 6/25
157/157 [==============================] - 1574s 10s/step - loss: 0.4459 - accuracy: 0.8261 - val_loss: 0.5860 - val_accuracy: 0.7760
Epoch 7/25
157/157 [==============================] - 1549s 10s/step - loss: 0.4365 - accuracy: 0.8291 - val_loss: 0.5509 - val_a

In [20]:
 n=15
xception_model.load_weights('xception_model_checkpoint_epoch_38.h5')
xception_model.fit(training_set, validation_data=validation_set, epochs=total_epochs + n,
                          validation_steps=len(validation_set), steps_per_epoch=len(training_set),
                          initial_epoch = 38,
                          callbacks= [checkpoint_callback]
         )

Epoch 39/40
157/157 [==============================] - 1428s 9s/step - loss: 0.1106 - accuracy: 0.9617 - val_loss: 0.5291 - val_accuracy: 0.8468
Epoch 40/40
118/157 [=====================>........] - ETA: 4:58 - loss: 0.1100 - accuracy: 0.9591

KeyboardInterrupt: 

In [11]:
xception_model.save("Lord_xception.h5")
from tensorflow.keras.models import load_model
modelcnn=load_model("Lord_xception.h5")

C:\Users\asus\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
model_path_35th_epoch = "xception_model_epoch_35.h5"
xception_model.save(model_path_35th_epoch)